In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
import seaborn as sns
sns.set_context('paper')

# Test plotting ellipses
from matplotlib.patches import Ellipse
def plot_ellipse(mu, cov, ax, std=1.0, facecolor='none', edgecolor='red'):
    a = cov[0, 0]
    b = cov[0, 1]
    c = cov[1, 1]
    lam1 = 0.5 * (a + c) + np.sqrt((0.5 * (a - c)) ** 2 + b ** 2)
    lam2 = 0.5 * (a + c) - np.sqrt((0.5 * (a - c)) ** 2 + b ** 2)
    if b == 0.0 and a >= c:
        theta = 0.0
    elif b == 0 and a < c:
        theta = 0.5 * np.pi
    else:
        theta = np.arctan2(lam1 - a, b)
    angle = 0.5 * 360.0 * theta / np.pi
    ell_radius_x = np.sqrt(lam1)
    ell_radius_y = np.sqrt(lam2)
    obj = Ellipse(mu, width=ell_radius_x * std, height=ell_radius_y * std,
                  angle=angle, facecolor=facecolor, edgecolor=edgecolor)
    ax.add_patch(obj)

# Hands-on Activity 20.1 - Kalman Filter for Object Tracking Example

## Objectives

+ Demonstrate the Kalman filter in the context of object tracking

Let's bring back the code from the hands-on activity of lecture 19.
We do not repeat the theoretical details.

In [ ]:
# The timestep
Dt = 0.5
# The mass
m = 1.0
# The variance for the process noise for position
epsilon = 1e-6
# The standard deviation for the process noise for velocity
sigma_q = 1e-2
# The standard deviation for the measurement noise for position
sigma_r = 0.1

# INITIAL CONDITIONS
# initial mean
mu0 = np.zeros((4,))
# initial covariance
V0 = np.array([0.1**2, 0.1**2, 0.1**2, 0.1**2]) * np.eye(4)

# TRANSITION MATRIX
A = np.array([[1.0, 0, Dt, 0],
              [0.0, 1.0, 0.0, Dt],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])
# CONTROL MATRIX
B = np.array([[0.0, 0.0],
              [0.0, 0.0],
              [Dt / m, 0.0],
              [0.0, Dt / m]])
# PROCESS COVARIANCE
Q = np.array([epsilon, epsilon, sigma_q ** 2, sigma_q ** 2]) * np.eye(4)
# EMISSION MATRIX
C = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0]])
# MEASUREMENT COVARIANCE
R = np.array([sigma_r ** 2, sigma_r ** 2]) * np.eye(2)

Generate a trajectory and observations:

In [ ]:
# The number of steps in the trajectory
num_steps = 50
# Space to store the trajectory (each state is 4-dimensional)
true_trajectory = np.ndarray((num_steps + 1, 4))
# Space to store the observations (each observation is 2-dimensional)
observations = np.ndarray((num_steps, 2))

# Sample the initial conditions
x0 = mu0 + np.sqrt(np.diag(V0)) * np.random.randn(4)
true_trajectory[0] = x0

# Pick a set of pre-determined forces to be applied to the object
# so that it does something interesting
force = .1
omega = 2.0 * np.pi / 5
times = Dt * np.arange(num_steps + 1)
us = np.zeros((num_steps, 2))
us[:, 0] = force * np.cos(omega * times[1:])
us[:, 1] = force * np.sin(omega * times[1:])

# Sample the trajectory
for n in range(num_steps):
    x = np.dot(A, true_trajectory[n]) + np.dot(B, us[n]) + np.sqrt(np.diag(Q)) * np.random.randn(4)
    true_trajectory[n+1] = x
    y = np.dot(C, x) + np.sqrt(np.diag(R)) * np.random.randn(2)
    observations[n] = y

We are not going to implement the filter from scratch.
We are going to use the Python module [FilterPy](https://filterpy.readthedocs.io/en/latest/).
This is not included in the default version of Google Colab.
You need to install it manually.
Here is how:

In [ ]:
!pip install filterpy

Now you should be able to load the library. Try the code below:

In [ ]:
from filterpy.kalman import KalmanFilter

To define the filter in ``FilterPy`` we need to give the dimensionality of the state space (``dim_x``) and the observations (``dim_z``).
Here is how:

In [ ]:
kf = KalmanFilter(dim_x=4, dim_z=2)

Now we need to make the filter aware of the various vectors and matrices specifing initial conditions, transitions, emissions, covariances, etc.
Note that ``FilterPy`` different notation than the one we use.
The correspondance of the notation is as follows:


| Name | This class  | ``FilterPy``  | 
|---|---|---|
|initial mean vector| $\mu_n$ | $x$ |
|initial covariance matrix| $V_n$ |  $P$ |
|state transition matrix| $A$  | $F$ |
|control matrix| $B$ | $B$ |
|process covariance matrix|  $Q$ | $Q$  |
|emission matrix | $C$ | $H$ |
|measurement covariance matrix | $R$  | $R$  |


This is how you can make the ``kf`` object aware of the various matrices:

In [ ]:
kf.x = mu0
kf.P = V0
kf.Q = Q
kf.R = R
kf.H = C
kf.F = A
kf.B = B

Now we can do filtering.
You can do one time step at a time.
This is what you would do if the data points were coming one by one:
Here is how:

In [ ]:
# DO NOT RERUN THIS WITHOUT RERUNNING THE INITIALIZATION CODE IN THE PREVIOUS
# CODE BLOCK
for n in range(1, num_steps):
    # Predict step (notice that you also need to pass the control (if there is any))
    kf.predict(u=us[n])
    # Make a figure one every ten time steps
    if n % 5 == 0:
        fig, ax = plt.subplots(dpi=150)
        # Plot the true trajectory up to this point
        ax.plot(true_trajectory[:n+1, 0], true_trajectory[:n+1, 1], 'md-', label='True trajectory')
        # Plot the observation we just made
        ax.plot(observations[n,0], observations[n,1], 'x', label='Observation')
        # Plot the mean of the position after the prediction step
        ax.plot(kf.x[0], kf.x[1], 'g.', label='After prediction step')
        # Plot a 95% credible elipse
        plot_ellipse(kf.x[:2], kf.P[:2, :2], ax, edgecolor='green')
    # Update step
    kf.update(observations[n])
    if n % 5 == 0:
        # Plot the mean of the position after the update step
        ax.plot(kf.x[0], kf.x[1], 'r.', label='After update step')
        # Plot a 95 % credible elipse
        plot_ellipse(kf.x[:2], kf.P[:2, :2], ax, std=3.0, edgecolor='red')
        plt.legend(loc='best')

Notice that the filter is very uncertain at the beginning.
Then it gradually becomes better and better.

The other way to run the filter is with all the data at once.
This is called a *batch* filter.
Here is how:

In [ ]:
# We need to reset the initial conditions
kf.x = mu0
kf.P = V0
# Here is the code that runs the batch:
res = kf.batch_filter(observations, us=us)

This returns the means and the covariances that you would have gotten at each timestep:

In [ ]:
means = res[0]
covs = res[1]

And here is an alternative way to visualize your uncertainty about the state at all times:

In [ ]:
y_labels = ['$x_1$', '$x_2$', '$x_3$', '$x_4$']

res_x = 1024
res_y = 768
dpi = 150
w_in = res_x / dpi
h_in = res_y / dpi
fig, ax = plt.subplots(4, 1, dpi=dpi)
fig.set_size_inches(w_in, h_in)

times = Dt * np.arange(num_steps + 1)

for j in range(4):
    ax[j].set_ylabel(y_labels[j])
ax[-1].set_xlabel('$t$ (time)')

for j in range(4):
    ax[j].plot(times[0:num_steps], true_trajectory[0:num_steps, j], 'b.-')
    ax[j].plot(times[1:num_steps], means[1:num_steps, j], 'r.')
    ax[j].fill_between(times[1:num_steps],
                            means[1:num_steps, j] - 2.0 * np.sqrt(covs[1:num_steps, j, j]),
                            means[1:num_steps, j] + 2.0 * np.sqrt(covs[1:num_steps, j, j]),
                            color='red', alpha=0.25)                
    if j < 2:
        ax[j].plot(times[1:num_steps], observations[:n, j], 'go')

## Questions

+ Rerun the code a couple of times to observe different trajectories.
+ Double the process noise variance $\sigma_q^2$. What happens?
+ Double the measurement noise variance $\sigma_r^2$. What happens?
+ Zero-out the control vector $\mathbf{u}_{0:n-1}$. What happens?